In [56]:
import os
import re
import pandas as pd


In [179]:
job_number = '45075917'
log_dir = 'log/'
job_re = re.compile(r'.*{}.*'.format(job_number))
scan_re = re.compile(r'.*expanded call.*(tfMRI_(?:CARIT)_(?:AP|PA)).*')
logfiles = [os.path.join(log_dir, f) for f in os.listdir(log_dir) if job_re.match(f)]

In [187]:
error_strings = {"F-test" : "F-test 2 isn't valid",
                "dtseries" : "ERROR: failed to open file",
                "csv" : "ls: cannot access"}
def error_info(error_strings, f):
    if type(error_strings) not in [dict, list]:
        raise TypeError("error_strings is not a list")
    if type(error_strings) is dict:
        error_strings = list(error_strings.values())
    errors = []
    scan = ''
    for line in f:
        if(any([e for e in error_strings if e in line])):
            errors.append(line)
        m = scan_re.match(line)
        if m:
            scan = m.group(1)
    return(errors, scan)
                
        
    return()
def error_types(error_strings, errors):
    if type(error_strings) not in [dict, list]:
        raise TypeError("error_strings is not a list")
    if type(error_strings) is dict:
        error_strings = list(error_strings.values())
    error_types = [0, 0, 0]
    for error in errors:
        ii = [i for i, e in enumerate(error_strings) if e in error]
        for i in ii:
            error_types[i] += 1
    return(error_types) 

In [188]:
errors = []
for logfile in logfiles:
    with open(logfile) as f:
        these_errors, scan_with_error = error_info(error_strings, f)
        these_error_types = error_types(error_strings, these_errors)
        errors.append([ logfile, len(these_errors), these_errors, *these_error_types, scan_with_error])

In [189]:
df = pd.DataFrame(errors, columns = ['logfile', 'n_errors', 'errors', *error_strings.keys(), 'scan'])
df[df.n_errors > 0]

,logfile,n_errors,errors,F-test,dtseries,csv,scan
2,log/hcp1st-45075917_4.out,1,[F-test 2 isn't valid - each included contrast...,1,0,0,tfMRI_CARIT_PA
17,log/hcp1st-45075917_37.out,1,[F-test 2 isn't valid - each included contrast...,1,0,0,tfMRI_CARIT_AP
23,log/hcp1st-45075917_21.out,1,[F-test 2 isn't valid - each included contrast...,1,0,0,tfMRI_CARIT_PA
27,log/hcp1st-45075917_26.out,1,[F-test 2 isn't valid - each included contrast...,1,0,0,tfMRI_CARIT_AP
44,log/hcp1st-45075917_23.out,1,[F-test 2 isn't valid - each included contrast...,1,0,0,tfMRI_CARIT_PA
...,...,...,...,...,...,...,...
742,log/hcp1st-45075917_743.out,3,[ERROR: failed to open file '/net/holynfs01/sr...,0,3,0,tfMRI_CARIT_PA
757,log/hcp1st-45075917_758.out,1,[F-test 2 isn't valid - each included contrast...,1,0,0,tfMRI_CARIT_AP
760,log/hcp1st-45075917_761.out,1,[F-test 2 isn't valid - each included contrast...,1,0,0,tfMRI_CARIT_AP
798,log/hcp1st-45075917_799.out,1,[ls: cannot access /ncf/hcp/data/CCF_HCD_STG_P...,0,0,1,
